In [1]:
import pandas as pd
import logging
import numpy as np
import importlib

import utils
import batch

importlib.reload(utils)
importlib.reload(batch)

<module 'batch' from '/Users/mattgc/code/match-importance/batch.py'>

In [2]:
inp = input("Level of logging (DEBUG, INFO, WARNING, ERROR, CRITICAL): ")
logging.basicConfig(level=getattr(logging, inp), force=True)

# Import Data

In [3]:
data = pd.read_csv('data/all_years/24_25.csv')
data["Date"] = pd.to_datetime(data["Date"], format="%d/%m/%Y")
data = data.sort_values(by=["Date", "Time"]).reset_index(drop=True)
t = len(data)
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA
0,E0,2024-08-16,20:00,Man United,Fulham,1,0,H,0,0,...,1.86,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08
1,E0,2024-08-17,12:30,Ipswich,Liverpool,0,2,A,0,0,...,2.05,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93
2,E0,2024-08-17,15:00,Arsenal,Wolves,2,0,H,1,0,...,2.02,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96
3,E0,2024-08-17,15:00,Everton,Brighton,0,3,A,0,1,...,1.87,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11
4,E0,2024-08-17,15:00,Newcastle,Southampton,1,0,H,1,0,...,1.87,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,E0,2024-12-04,19:30,Southampton,Chelsea,1,5,A,1,3,...,2.07,1.86,2.07,1.87,2.07,1.90,2.01,1.86,2.07,1.92
136,E0,2024-12-04,20:15,Arsenal,Man United,2,0,H,0,0,...,2.07,1.86,2.07,1.86,2.07,1.93,1.99,1.87,2.06,1.93
137,E0,2024-12-04,20:15,Aston Villa,Brentford,3,1,H,3,0,...,1.85,2.08,1.85,2.08,1.88,2.08,1.84,2.02,1.92,2.07
138,E0,2024-12-05,19:30,Fulham,Brighton,3,1,H,1,0,...,1.94,1.99,1.94,1.98,1.96,2.00,1.92,1.96,1.97,2.01


In [4]:
remaining = pd.read_csv('data/all_years/24_25_remaining.csv')
remaining

,HomeTeam,AwayTeam
0,Everton,Liverpool
1,Aston Villa,Southampton
2,Brentford,Newcastle
3,Crystal Palace,Man City
4,Man Utd,Nott'm Forest
...,...,...
235,Newcastle United,Everton
236,Nottingham Forest,Chelsea
237,Southampton,Arsenal
238,Spurs,Brighton


In [5]:
# Concatenate remaining data to the original data
data = pd.concat([data, remaining], ignore_index=True)
data["TeamEloWinProb"] = 0
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFECAHH,BFECAHA,TeamEloWinProb
0,E0,2024-08-16,20:00,Man United,Fulham,1.0,0.0,H,0.0,0.0,...,2.07,1.83,2.11,1.88,2.11,1.82,2.05,1.90,2.08,0
1,E0,2024-08-17,12:30,Ipswich,Liverpool,0.0,2.0,A,0.0,0.0,...,1.88,2.04,1.90,2.20,2.00,1.99,1.88,2.04,1.93,0
2,E0,2024-08-17,15:00,Arsenal,Wolves,2.0,0.0,H,1.0,0.0,...,1.91,2.00,1.90,2.05,1.93,1.99,1.87,2.02,1.96,0
3,E0,2024-08-17,15:00,Everton,Brighton,0.0,3.0,A,0.0,1.0,...,2.06,1.86,2.07,1.92,2.10,1.83,2.04,1.88,2.11,0
4,E0,2024-08-17,15:00,Newcastle,Southampton,1.0,0.0,H,1.0,0.0,...,2.06,1.88,2.06,1.89,2.10,1.82,2.05,1.89,2.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,NaN,NaT,NaN,Newcastle United,Everton,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
376,NaN,NaT,NaN,Nottingham Forest,Chelsea,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
377,NaN,NaT,NaN,Southampton,Arsenal,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
378,NaN,NaT,NaN,Spurs,Brighton,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [6]:
to_predict = pd.read_csv('data/epl-test.csv')
to_predict

,Date,HomeTeam,AwayTeam
0,01-Feb-25,AFC Bournemouth,Liverpool
1,01-Feb-25,Arsenal,Man City
2,01-Feb-25,Brentford,Spurs
3,01-Feb-25,Chelsea,West Ham
4,01-Feb-25,Everton,Leicester City
5,01-Feb-25,Ipswich Town,Southampton
6,01-Feb-25,Man Utd,Crystal Palace
7,01-Feb-25,Newcastle,Fulham
8,01-Feb-25,Nottingham Forest,Brighton
9,01-Feb-25,Wolves,Aston Villa


In [7]:
t_k = data[(data["HomeTeam"] == "AFC Bournemouth") & (data["AwayTeam"] == "Liverpool")].index[0]
print(t_k)
data.iloc[t_k]

230


Div                           NaN
Date                          NaT
Time                          NaN
HomeTeam          AFC Bournemouth
AwayTeam                Liverpool
                       ...       
AvgCAHH                       NaN
AvgCAHA                       NaN
BFECAHH                       NaN
BFECAHA                       NaN
TeamEloWinProb                  0
Name: 230, Length: 121, dtype: object

# Run a single simulation

In [8]:
standings = utils.calculate_standings(data, 0, t)
standings

,Team,Points
0,Liverpool,36
1,Arsenal,28
2,Chelsea,28
3,Man City,26
4,Brighton,23
5,Fulham,22
6,Aston Villa,22
7,Nott'm Forest,22
8,AFC Bournemouth,21
9,Newcastle,20


In [9]:
simulated_standings = utils.run_simulation(data, standings, t, t_k).sort_values(by="Points", ascending=False).reset_index(drop=True)
simulated_standings

,Team,Points
0,Liverpool,65
1,Fulham,65
2,Arsenal,64
3,Chelsea,62
4,Aston Villa,62
5,Man City,58
6,Brentford,58
7,Brighton,57
8,West Ham,54
9,AFC Bournemouth,51


In [10]:
simulated_standings

,Team,Points
0,Liverpool,65
1,Fulham,65
2,Arsenal,64
3,Chelsea,62
4,Aston Villa,62
5,Man City,58
6,Brentford,58
7,Brighton,57
8,West Ham,54
9,AFC Bournemouth,51


In [11]:
utils.calculate_outcomes(simulated_standings, "AFC Bournemouth", "Liverpool")

(0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1)

# Predict match importance for match $t+k$ with repeated simulations

In [12]:
data.loc[t_k, ["HI", "AI"]] = utils.calculate_match_importance( # type: ignore
    data,
    standings,
    t_k,
    t,
    nruns=100,
)

In [13]:
data.iloc[t_k]

Div                           NaN
Date                          NaT
Time                          NaN
HomeTeam          AFC Bournemouth
AwayTeam                Liverpool
                       ...       
BFECAHH                       NaN
BFECAHA                       NaN
TeamEloWinProb                  0
HI                           0.04
AI                           0.02
Name: 230, Length: 123, dtype: object

# Backfill match importance in master file

In [14]:
master = pd.read_csv('data/michal-master.csv').drop_duplicates()
master["Date"] = pd.to_datetime(master["Date"], format="%Y-%m-%d").dt.strftime("%d-%b-%y")
master

,Date,HomeTeam,AwayTeam,HomeElo,AwayElo,HomeRank,AwayRank,HomePlayerRatingCount,HomePlayerRatingMean,HomePlayerRatingStd,AwayPlayerRatingCount,AwayPlayerRatingMean,AwayPlayerRatingStd,TeamEloWinProb,PlayerEloDrawProb,PlayerEloWinProb,HomeEloMomentum,AwayEloMomentum,HomeRankMomentum,AwayRankMomentum
0,05-Jan-10,Stoke,Fulham,1621.513306,1739.930908,15.0,8.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.335889,0.330000,0.330000,0.000000,0.000000,0.0,0.0
1,09-Jan-10,Arsenal,Everton,1877.290039,1703.894897,3.0,9.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.730693,0.330000,0.330000,0.000000,0.000000,0.0,0.0
2,09-Jan-10,Birmingham,Man United,1653.174805,1927.595947,11.0,2.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.170839,0.330000,0.330000,0.000000,0.000000,0.0,0.0
3,11-Jan-10,Man City,Blackburn,1745.521851,1642.066650,7.0,13.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.644634,0.330000,0.330000,0.000000,0.000000,0.0,0.0
4,16-Jan-10,Chelsea,Sunderland,1945.430176,1606.012695,1.0,18.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.875865,0.330000,0.330000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,19-May-24,Crystal Palace,Aston Villa,1742.572754,1787.570190,9.0,7.0,14.0,6.966429,0.327689,14.0,6.925714,0.291908,0.435603,0.269324,0.475556,-48.135376,41.060791,6.0,-2.0
5800,19-May-24,Liverpool,Wolves,1898.975098,1681.206665,3.0,16.0,17.0,7.169412,0.293948,15.0,6.921333,0.225325,0.777918,0.209490,0.643965,19.503418,38.082886,0.0,-3.0
5801,19-May-24,Luton,Fulham,1577.058716,1707.895386,22.0,13.0,16.0,6.679375,0.278242,15.0,6.836000,0.187111,0.320134,0.279518,0.298134,39.452759,11.820312,0.0,0.0
5802,19-May-24,Man City,West Ham,2044.003174,1728.072998,1.0,11.0,13.0,7.429231,0.306104,12.0,6.951667,0.301160,0.860403,0.117857,0.817173,-3.334229,51.946533,0.0,-3.0


In [29]:
all_years = pd.concat(
    [
        pd.read_csv(f"data/all_years/{season}.csv")
        for season in [
            f"{i}_{i+1}"
            for i in range(17, 24)
        ]
    ]
)
all_years["Date"] = pd.to_datetime(all_years["Date"], format="%d/%m/%Y")
all_years = all_years.sort_values(by=["Date"]).reset_index(drop=True)
all_years["Date"] = all_years["Date"].dt.strftime("%d-%b-%y")
all_years

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11-Aug-17,Arsenal,Leicester,4,3,H,2,2,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,12-Aug-17,Brighton,Man City,0,2,A,0,0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,12-Aug-17,Chelsea,Burnley,2,3,A,0,3,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,12-Aug-17,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,12-Aug-17,Everton,Stoke,1,0,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,E0,19-May-24,Brighton,Man United,0,2,A,0,0,D,...,3.46,0.00,2.06,1.87,2.04,1.89,2.12,1.89,2.04,1.84
2656,E0,19-May-24,Brentford,Newcastle,2,4,A,0,3,A,...,3.24,0.25,1.93,2.00,1.94,1.98,1.97,2.02,1.92,1.94
2657,E0,19-May-24,Arsenal,Everton,2,1,H,1,1,D,...,3.09,-2.00,2.04,1.89,2.02,1.89,2.05,1.91,1.98,1.89
2658,E0,19-May-24,Burnley,Nott'm Forest,1,2,A,0,2,A,...,2.50,0.25,1.85,2.08,1.85,2.08,1.90,2.09,1.85,2.02


In [30]:
# Add team ELO feature from master to all_years
match_importance = all_years.merge(
    master,
    on=["Date", "HomeTeam", "AwayTeam"],
    suffixes=("", "_master"),
)
match_importance[["HI", "AI"]] = np.nan
match_importance

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AwayPlayerRatingStd,TeamEloWinProb,PlayerEloDrawProb,PlayerEloWinProb,HomeEloMomentum,AwayEloMomentum,HomeRankMomentum,AwayRankMomentum,HI,AI
0,E0,11-Aug-17,Arsenal,Leicester,4,3,H,2,2,D,...,0.313912,0.680436,0.330000,0.330000,-42.665039,2.786133,1.0,0.0,NaN,NaN
1,E0,12-Aug-17,Brighton,Man City,0,2,A,0,0,D,...,0.344408,0.163951,0.330000,0.330000,0.000000,-18.119751,0.0,1.0,NaN,NaN
2,E0,12-Aug-17,Chelsea,Burnley,2,3,A,0,3,A,...,0.314456,0.833734,0.330000,0.330000,-30.350952,3.496094,0.0,-1.0,NaN,NaN
3,E0,12-Aug-17,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,0.219225,0.723831,0.330000,0.330000,0.091187,0.000000,-2.0,0.0,NaN,NaN
4,E0,12-Aug-17,Everton,Stoke,1,0,H,1,0,H,...,0.209960,0.623467,0.330000,0.330000,19.391602,3.265503,0.0,-1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,E0,19-May-24,Brighton,Man United,0,2,A,0,0,D,...,0.269488,0.461142,0.277544,0.287346,17.913330,33.503296,0.0,-1.0,NaN,NaN
2656,E0,19-May-24,Brentford,Newcastle,2,4,A,0,3,A,...,0.322628,0.391223,0.277362,0.286444,-1.940796,15.200317,0.0,2.0,NaN,NaN
2657,E0,19-May-24,Arsenal,Everton,2,1,H,1,1,D,...,0.235679,0.793287,0.200926,0.662440,-11.984131,-19.581909,0.0,2.0,NaN,NaN
2658,E0,19-May-24,Burnley,Nott'm Forest,1,2,A,0,2,A,...,0.183726,0.473702,0.281422,0.311181,14.532104,22.477661,1.0,1.0,NaN,NaN


In [31]:
# Length of a season is 380 games
for i in range(380, len(match_importance)+1, 380):
    match_importance = batch.calculate_match_importance(
        data=match_importance,
        tstart=i-380,
        tend=i-1,
        nruns=10
    )

Calculating match importance for the season, 0 to 379
Match 0 done: Arsenal vs Leicester, 0.19999999999999996, 0.1
Match 1 done: Brighton vs Man City, 0.0, 0.20000000000000007
Match 2 done: Chelsea vs Burnley, 0.19999999999999996, 0.1
Match 3 done: Crystal Palace vs Huddersfield, 0.0, 0.0
Match 4 done: Everton vs Stoke, 0.0, 0.0
Match 5 done: Southampton vs Swansea, 0.0, 0.0
Match 6 done: Watford vs Liverpool, 0.0, 0.1
Match 7 done: West Brom vs Bournemouth, 0.0, 0.0
Match 8 done: Man United vs West Ham, 0.09999999999999998, 0.0
Match 9 done: Newcastle vs Tottenham, 0.0, 0.0
Match 10 done: Swansea vs Man United, 0.0, 0.19999999999999998
Match 11 done: Stoke vs Arsenal, 0.0, 0.3
Match 12 done: Southampton vs West Ham, 0.0, 0.0
Match 13 done: Burnley vs West Brom, 0.1, 0.0
Match 14 done: Leicester vs Brighton, 0.1, 0.0
Match 15 done: Bournemouth vs Watford, 0.0, 0.0
Match 16 done: Liverpool vs Crystal Palace, 0.1, 0.0
Match 17 done: Huddersfield vs Newcastle, 0.0, 0.0
Match 18 done: Tott

In [32]:
match_importance

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AwayPlayerRatingStd,TeamEloWinProb,PlayerEloDrawProb,PlayerEloWinProb,HomeEloMomentum,AwayEloMomentum,HomeRankMomentum,AwayRankMomentum,HI,AI
0,E0,11-Aug-17,Arsenal,Leicester,4,3,H,2,2,D,...,0.313912,0.680436,0.330000,0.330000,-42.665039,2.786133,1.0,0.0,0.2,0.1
1,E0,12-Aug-17,Brighton,Man City,0,2,A,0,0,D,...,0.344408,0.163951,0.330000,0.330000,0.000000,-18.119751,0.0,1.0,0.0,0.2
2,E0,12-Aug-17,Chelsea,Burnley,2,3,A,0,3,A,...,0.314456,0.833734,0.330000,0.330000,-30.350952,3.496094,0.0,-1.0,0.2,0.1
3,E0,12-Aug-17,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,0.219225,0.723831,0.330000,0.330000,0.091187,0.000000,-2.0,0.0,0.0,0.0
4,E0,12-Aug-17,Everton,Stoke,1,0,H,1,0,H,...,0.209960,0.623467,0.330000,0.330000,19.391602,3.265503,0.0,-1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2655,E0,19-May-24,Brighton,Man United,0,2,A,0,0,D,...,0.269488,0.461142,0.277544,0.287346,17.913330,33.503296,0.0,-1.0,0.0,0.0
2656,E0,19-May-24,Brentford,Newcastle,2,4,A,0,3,A,...,0.322628,0.391223,0.277362,0.286444,-1.940796,15.200317,0.0,2.0,0.0,0.0
2657,E0,19-May-24,Arsenal,Everton,2,1,H,1,1,D,...,0.235679,0.793287,0.200926,0.662440,-11.984131,-19.581909,0.0,2.0,0.0,0.0
2658,E0,19-May-24,Burnley,Nott'm Forest,1,2,A,0,2,A,...,0.183726,0.473702,0.281422,0.311181,14.532104,22.477661,1.0,1.0,0.0,0.0


In [35]:
match_importance.to_csv('data/match_importance_10.csv', index=False)